In [1]:
import pandas as pd
import numpy as  np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train=train.drop('Loan_ID',1)
loan_id=test.Loan_ID
test=test.drop('Loan_ID',1)

In [4]:
#Imputing missing values for both train and test
train.fillna(-999, inplace=True)
test.fillna(-999,inplace=True)


#Creating a training set for modeling and validation set to check model performance
y = train.Interest_Rate
X = train.drop(['Interest_Rate'], axis=1)
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [5]:
from sklearn.preprocessing import LabelEncoder
categorical=list(train.columns[categorical_features_indices])
for var in categorical:
    lb = LabelEncoder()
    full_var_data =X.append(test)[var].astype('str')
    temp = lb.fit_transform(np.array(full_var_data))
    X[var] = lb.transform(np.array(X[var]).astype('str'))
    test[var] = lb.transform(np.array(test[var]).astype('str'))
    
dic={1:0,2:1,3:2}
y=y.apply(lambda x:dic[x])

In [6]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1234)

In [14]:
gbm = lgb.LGBMClassifier(objective='Multiclass',
                        num_leaves=100,
                        learning_rate=0.1,
                        n_estimators=200,num_class=3,max_depth=7,max_bin=200)

In [17]:
from sklearn import cross_validation
X.shape

(164309, 14)

In [16]:
cv = cross_validation.KFold(len(X), n_folds=5)
from sklearn.model_selection import cross_val_score


array([ 0.70629583,  0.70506969,  0.7125251 ,  0.70585192,  0.70506071])

In [30]:
cross_val_score(gbm,X,y, cv=3,scoring='accuracy')

array([ 0.70539154,  0.70671901,  0.70420684])

In [23]:
gbm.fit(X,y)

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
        max_bin=200, max_depth=7, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=200, nthread=-1, num_class=3,
        num_leaves=100, objective='multiclass', reg_alpha=0, reg_lambda=0,
        seed=0, silent=True, subsample=1, subsample_for_bin=50000,
        subsample_freq=1)

In [24]:
gbm_prob=gbm.predict_proba(test)

In [25]:
gbm_prob[0]

array([ 0.16405385,  0.74746007,  0.08848607])

In [26]:
data=pd.DataFrame(gbm_prob)

In [29]:
data.to_csv('lgb.csv',index=False)

In [33]:
gbm.feature_importances_

array([8679, 2941, 2900, 1458, 6951, 1709, 3188, 8024, 2199, 4819, 3617,
       6094, 1009,  522])

In [ ]:
10.062625278836638, 43.610347804442, 1.247512124987842,
2.5981804748929216, 3.0014023676223336, 6.562985190807639,
6.685612517117853, 5.668568473228082, 7.000167401745386, 
4.165811530490307, 0.8253362874834277, 3.4365669750350936,
1.9357110638937611, 3.1991725094167105